In [3]:
!tar -xvf  'data.tar.gz' -C '.'

data/
data/task1/
data/task1/ts.csv
data/task2/
data/task2/df.csv
data/task2/ts_0.csv
data/task2/ts_1.csv
data/task2/ts_10.csv
data/task2/ts_100.csv
data/task2/ts_101.csv
data/task2/ts_102.csv
data/task2/ts_103.csv
data/task2/ts_104.csv
data/task2/ts_105.csv
data/task2/ts_106.csv
data/task2/ts_107.csv
data/task2/ts_108.csv
data/task2/ts_109.csv
data/task2/ts_11.csv
data/task2/ts_110.csv
data/task2/ts_111.csv
data/task2/ts_112.csv
data/task2/ts_113.csv
data/task2/ts_114.csv
data/task2/ts_115.csv
data/task2/ts_116.csv
data/task2/ts_117.csv
data/task2/ts_118.csv
data/task2/ts_119.csv
data/task2/ts_12.csv
data/task2/ts_120.csv
data/task2/ts_121.csv
data/task2/ts_122.csv
data/task2/ts_123.csv
data/task2/ts_124.csv
data/task2/ts_125.csv
data/task2/ts_126.csv
data/task2/ts_127.csv
data/task2/ts_128.csv
data/task2/ts_129.csv
data/task2/ts_13.csv
data/task2/ts_130.csv
data/task2/ts_131.csv
data/task2/ts_132.csv
data/task2/ts_133.csv
data/task2/ts_134.csv
data/task2/ts_135.csv
data/task2/ts_136.

In [65]:
import os 
import pandas as pd
import numpy as np
from fbprophet import Prophet 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [18]:
files_path = '/content/data/task2'
list_files = os.listdir(files_path)
list_files = [file for file in list_files if file.startswith('ts_')]

time_series = []
for file_name in list_files:
    time_series.append(pd.read_csv(os.path.join(files_path, file_name), names=['ds', 'y']))

df_ts = pd.concat(time_series, axis=1)

In [64]:
history = []
for i, series in enumerate(time_series):
    series = series.fillna(series.y.mean()) 
    # возьмем небольшой сэмпл, большинство временных рядов имеют около 1к точек, попытаемся сделать прогноз на примерно 2 недели
    train_size = int(len(series) * 0.95) 
    train, test = series.iloc[:train_size], series.iloc[train_size:]
    m = Prophet()
    m.fit(train)
    future = pd.DataFrame(test.ds.to_list())
    future.columns = ['ds']
    forecast = m.predict(future)
    history.append(pd.DataFrame(data={'forecast': forecast.yhat.values, 
                                 'actual' : test.y.values,
                                 'date' : test.ds.values, 
                                 'ts_num' : [i] * len(test)}))

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [81]:
history_df = pd.concat(history)
history_df['mae'] = np.abs(history_df.forecast - history_df.actual) 

#MAE старая добрая проверенная годами метрика, универсальна для множества временных рядов 
samples = np.random.choice(history_df.ts_num.unique(), 25)


fig = make_subplots(rows=5, cols=5)
i, j = 1, 1
for sample in samples:
    sub_df = history_df[history_df.ts_num == sample]
    fig.add_trace(go.Scatter(x=sub_df.date, y=sub_df.forecast, name=f'Forecast for ts_{sample}, MAE={sub_df.mae.mean()}'), row=i, col=j)
    fig.add_trace(go.Scatter(x=sub_df.date, y=sub_df.actual, name=f'Actual for ts_{sample}'), row=i, col=j)
    j += 1 
    if j % 6 == 0:
        i += 1 
        j = 1

fig.show()
